In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape

In [ ]:
mun_ser = pd.read_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\municipios_sertao.csv').drop(columns='Unnamed: 0')

In [ ]:
solos = pd.read_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\brasil_solos_5m_20201104.csv',sep=',')
solos['geometry'] = solos['the_geom'].apply(wkt.loads)
solos = solos[['LEG_SINOT', 'geometry']]
solos = gpd.GeoDataFrame(solos, geometry='geometry')

In [ ]:
lim_muns = pd.read_json(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\municipios-poligonos.json')
lim_muns = lim_muns[lim_muns['municipioCodigo'].isin(mun_ser['IBGE7'].unique())]
lim_muns['poligono'] = [str(polygon) for polygon in lim_muns['poligono']]
lim_muns['geometry'] = lim_muns['poligono'].apply(lambda x: shape(eval(x)))
lim_muns = lim_muns[['municipioCodigo', 'geometry']]
lim_muns.columns = ['IBGE7', 'geometry']
lim_muns = gpd.GeoDataFrame(lim_muns, geometry='geometry')

In [ ]:
intersecao_gdf = gpd.overlay(solos, lim_muns, how='intersection')
intersecao_gdf['AreaSoloHectares'] = intersecao_gdf['geometry'].area / 10000  # Converter para hectares
merged_data = lim_muns.merge(intersecao_gdf.groupby(['IBGE7', 'LEG_SINOT'])['AreaSoloHectares'].sum().reset_index(), on='IBGE7', how='left')
merged_data.rename(columns={'LEG_SINOT': 'SOLO', 'AreaSoloHectares': 'AREA_TOTAL'}, inplace=True)
tabela_final = merged_data[['IBGE7', 'SOLO', 'AREA_TOTAL']]
tabela_final.to_csv(r'C:\Users\User\Documents\GitHub\BCG2023\Dados\Views\solos_municipios.json')

In [ ]:
tabela_final[tabela_final['']]